In [1]:
import os
import pandas as pd

In [2]:
import re

In [3]:
from configparser import ConfigParser
cfg = ConfigParser()
cfg.read("config.cfg")

['config.cfg']

In [4]:
from tqdm._tqdm_notebook import tqdm_notebook as tqdm # we manually import the notebook submodule as the normal one struggles with jupyterlab
tqdm.pandas() # this enables us to use progress_apply instead of apply

### Load previous data

In [5]:
df = pd.read_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]001_dataframe_full.p"))

In [6]:
df.head()

,comment,dataset_file,model,note,page_id,page_ns,page_title,timestamp,user_id,user_ip,user_name
0,/* wbsetdescription-add:1|ar */ مؤرخ ألماني,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101383,0,Q98773,2017-06-23 19:37:19+00:00,2437879,,Benseid Seid
1,"/* wbsetlabel-add:1|sl */ Gerhard Raff, #quick...",wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101383,0,Q98773,2018-01-29 13:38:28+00:00,23475,,Sporti
2,/* wbsetsitelink-add:1|enwiki */ Wolfram Wette,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-01-04 17:08:50+00:00,17848,,Assayer
3,/* wbsetreference-add:2| */ [[Property:P937]]:...,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-06-06 08:21:58+00:00,38324,,Olaf Kosinsky
4,/* wbcreateclaim-create:1| */ [[Property:P1412...,wikidatawiki-20190101-pages-meta-history1.xml-...,wikibase-item,,101384,0,Q98774,2017-06-06 14:59:40+00:00,38324,,Olaf Kosinsky


### Load manual label matchers

In [7]:
df_manual_labels = pd.read_csv(os.path.join(cfg.get("directory", "dataset"), cfg.get("dataset", "wikidata_aux"), cfg.get("dataset", "wikidata_manual_labels")), engine='python')
df_manual_labels['match'] = df_manual_labels['match'].apply(lambda x: " " + x.strip() + " ")

In [8]:
comment_regex = re.compile(r"\/\* (.*):(\d*)\|(.*?)( \*\/|\|)")

def split_comment(comment):
    if comment == "" or pd.isna(comment):
        base_action = ""
        digit = ""
        language_code = ""
    else:
        match = re.match(comment_regex, comment)
        if match is not None:
            base_action = match.group(1).strip()
            digit = match.group(2).strip()
            language_code = match.group(3).strip()
        else:
            # the re did not match. let's see if we can match anything from the manual labels
            comment_lower = comment.lower().replace(":", "").strip()
            comment_lower = " " + comment_lower + " "
            matches = df_manual_labels.loc[df_manual_labels['match'].apply(lambda x: x in comment_lower)] # look for words
            if len(matches) > 0:
                base_action = matches['label'].value_counts().idxmax() # take the most common label from the matches
            else:
                base_action = ""
            digit = ""
            language_code = ""
    return {
        "action_base": base_action,
        "action_digit": digit,
        "action_language": language_code
    }

### Extract labels

In [ ]:
df_actions = pd.concat([df, pd.DataFrame.from_records(df['comment'].progress_apply(split_comment))], axis=1)

In [ ]:
df_actions.head()

### Remove unmatched revisions

In [ ]:
print("Pre unmatched comments", len(df_actions))
df_actions = df_actions.loc[~((df_actions["action_base"] == "") & (df_actions['comment'] != ""))].reset_index(drop=True)
print("Post unmatched comments", len(df_actions))

In [ ]:
num_labels = len(df_actions['action_base'].unique())
print("Num Labels {n}".format(n=num_labels))

### Remove all users with a low number of changes (below threshold)

In [ ]:
change_threshold = cfg.getint("preprocessing", "min_num_changes")

In [ ]:
df_num_changes = df_actions["user_id"].value_counts()

In [ ]:
print("# Revisions before: {n}".format(n=len(df_actions)))
df_actions = df_actions.loc[~df_actions['user_id'].isin(df_num_changes.loc[df_num_changes < change_threshold].index)].reset_index(drop=True)
print("# Revisions before: {n}".format(n=len(df_actions)))

### Store Data

In [ ]:
df_actions.to_pickle(os.path.join(cfg.get("directory", "exchange"), "[wikidata]002_extract_actions.p"))